# Сбор датасета

In [200]:
import requests
from bs4 import BeautifulSoup
from operator import attrgetter, itemgetter
from tqdm.notebook import tqdm
import itertools
import pandas as pd
import re
import time
import sys
import gzip
import json
import codecs
import pandas as pd

from itertools import chain 
from contextlib import ExitStack 
import os

from typing import Generator, Dict, Any
from multiprocessing.dummy import Pool, Queue

In [241]:
# ROOT = "https://shop.relod.ru"
# CAT_PATH = "https://shop.relod.ru/catalog-products/4577"

ROOT = "https://www.poetryfoundation.org"
AUTHORS_PATH = "https://www.poetryfoundation.org/poets/browse" # '#page=1&sort_by=recently_added&preview=0'
POEM_SEP = '@' * 20
AUTHOR_SEP = '$' * 20

Вспомогательные функции:

In [202]:
def get_page(url, params=None, n_attempts=10, t_sleep=2):
    assert n_attempts >= 1 and t_sleep > 0
    for i in range(n_attempts):
        page = requests.get(url, params=params)
        if page.ok:
            break
        
        time.sleep(t_sleep)
    if not page.ok:
        return (page.ok, page.status_code)
    return (page.ok, page)

In [203]:
def get_poets_from_html(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    tags = soup.select('li div div h2 a')
    
    if len(tags) == 0:
        print("Poems: No tags a with required parameters", file=sys.stderr)
        return []
    
    links = [tag['href'] for tag in tags]
        
    return links


def get_poems_from_html(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    tags = soup.select('li div div h3 a')
    
#     if len(tags) == 0:
#         print("Poems: No tags a with required parameters", file=sys.stderr)
#         return []
    
    links = [tag['href'] for tag in tags]
        
    return links

In [204]:
params = {
        "sort_by" : "recently_added",
        "preview" : 0,
        "page" : 1
    }
ok, page = get_page(AUTHORS_PATH, params)
poets_url = get_poets_from_html(page.text)

ok, page = get_page(poets_url[3])
for poem in get_poems_from_html(page.text):
    print(poem)

https://www.poetryfoundation.org/poetrymagazine/poems/23078/alder-red
https://www.poetryfoundation.org/poetrymagazine/poems/23690/the-black-panther-after-every-war
https://www.poetryfoundation.org/poetrymagazine/poems/27082/the-bridge-56d21102e5059
https://www.poetryfoundation.org/poetrymagazine/poems/21428/departure-56d20ca8de315
https://www.poetryfoundation.org/poetrymagazine/poems/35834/half-back-in-the-open
https://www.poetryfoundation.org/poetrymagazine/poems/35832/line-plunger
https://www.poetryfoundation.org/poetrymagazine/poems/23079/little-we-knew
https://www.poetryfoundation.org/poetrymagazine/poems/23689/new-york-subway
https://www.poetryfoundation.org/poetrymagazine/poems/21429/open-high-windows
https://www.poetryfoundation.org/poetrymagazine/poems/35833/the-passer
https://www.poetryfoundation.org/poetrymagazine/poems/23080/seagull-city


## Реализуем обкачку сайта

In [223]:
# '#page=1&sort_by=recently_added&preview=0'
def get_links_from_page(page_num):
    params = {
        "sort_by" : "recently_added",
        "preview" : 0,
        "page" : page_num
    }
    ok, page = get_page(AUTHORS_PATH, params)
    if not ok:
        print("Page with base {} and params {} couldn't "
              "be loaded. Status code: {}".format(AUTHORS_PATH, params, page), file=sys.stderr)
        return {}
    
    poets_url = get_poets_from_html(page.text)
    poems_dict = {}
    skipped = 0
    for poet_url in poets_url:
        ok, page = get_page(poet_url)
        poems = get_poems_from_html(page.text)
        if len(poems) <= 5:
            skipped += 1
        else:
            poems_dict[poet_url] = poems
    
    # print('Skipped {} poets(less than 5 poems) from {}'.format(skipped, len(poets_url)), file=sys.stderr)
    return poems_dict

In [224]:
def get_all_links(start_page_num=1, page_count=250):
    poems_dict = {}
    for page_num in tqdm(range(start_page_num, start_page_num + page_count)):
        poems_dict.update( get_links_from_page(page_num) )
    
    return poems_dict

In [230]:
queue = Queue()

for key in range(1, 151):
    queue.put(key)

queue.empty()

False

In [231]:
links = {}

In [232]:
def process_load_links(i):
    
    while not queue.empty():
        key = queue.get()
        res = get_links_from_page(key)
        
        with lock:
            links.update( res )
            pbar.update(1)

In [233]:
with Pool(processes=4) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_load_links, range(pool._processes))

pool.join()

# links = get_all_links(1, 100)
len(links)

980

## Извлечение текста

In [235]:
def get_text(soup):
    tag = soup.find_all('script', attrs={'type' : 'application/ld+json'})
    if len(tag) == 0:
        return None

    tag = tag[0].string
    ans = re.findall(r'"text":[^}]*}', tag)
    
    if (len(ans) != 1):
        if len(ans) == 1:
            print('Unexpected len: {}'.format(len(ans)), file=sys.stderr)
        return None
    
    text = re.sub(r'"text":"', '', ans[0])
    text = re.sub(r'"}', '', text)
    
    return text

In [236]:
def process_page(url):
    ok, page = get_page(url)
    if not ok:
        return ok, page
    
    soup = BeautifulSoup(page.text, 'html.parser')
    info = get_text(soup)
    if info is None:
        ok = False
        
    return ok, info

In [247]:
queue = Queue()

for key_link in links:
    queue.put(key_link)

queue.empty()

False

In [248]:
for key_link in links:
    print(links[key_link])
    break

['https://www.poetryfoundation.org/poems/151394/aphasia', 'https://www.poetryfoundation.org/poems/52834/blue-56d2319f16cad', 'https://www.poetryfoundation.org/poems/52835/chant-56d2319f84f6e', 'https://www.poetryfoundation.org/poems/52833/dog-woman', 'https://www.poetryfoundation.org/poems/52797/durban-south-africa-some-notations-of-value', 'https://www.poetryfoundation.org/poems/52799/hanging-in-egypt-with-breyten-breytenbach', 'https://www.poetryfoundation.org/poems/52832/in-the-middle-of-dinner', 'https://www.poetryfoundation.org/poems/52798/the-new-religion', 'https://www.poetryfoundation.org/poems/52800/war-widow', 'https://www.poetryfoundation.org/poems/151393/well-meant']


In [249]:
def process_page_wrapper(i):
    
    with open('data/poems{:05d}.txt'.format(i), mode='w') as txt:
        while not queue.empty():
            key = queue.get()
            for url in links[key]:
                try:
                    ok, record = process_page(url)
                except:
                    ok, record = False, None
                if ok:
                    print(record + POEM_SEP, file=txt)
            
            print(AUTHOR_SEP, file=txt)

            with lock:
                pbar.update(1)

In [250]:
with Pool(processes=4) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

pool.join()

TypeError: expected string or bytes-like object

## Структурируем информацию

In [251]:
from datetime import timedelta, datetime
from random import randrange

In [352]:
def get_random_dates(count, start_date = datetime(2015, 5, 27, 12, 0)):
    
    return sorted([start_date + timedelta(hours=randrange(1, 35051)) for i in range(count)])


def get_dataframe(data_dir, min_poems_count=5, novelty_del=5):
    
    df = pd.DataFrame(columns=['time', 'content', 'author_id', 'novel'])
    lib = []
    
    for filename in os.listdir(data_dir):
        if not filename.endswith('.txt'):
            continue

        with open(data_dir + filename, 'rb') as fin:
            author_poems = fin.read().decode("utf-8").split(AUTHOR_SEP)
            # author_poems = author_poems
            for author_poem in author_poems:
                if len( author_poem.strip() ) != 0:
                    poems = author_poem.split(POEM_SEP)
                    poems = list(filter(lambda poem: len(poem.strip()) > 0, poems ))
                    if len(poems) >= min_poems_count:
                        lib.append(poems)
    
    for author_id, poems in enumerate(lib):
        novel_count = np.clip(len(poems) // novelty_del, 1, 5)
        frame = {
            'time' : get_random_dates(len(poems)),
            'content' : poems,
            'author_id' : [author_id] * len(poems),
            'novel' : [1] * novel_count + [0] * (len(poems) - novel_count)
        }
        frame = pd.DataFrame.from_dict(frame)
        
        df = pd.concat((df, frame), axis=0, ignore_index=True)
    
    return df 

In [362]:
min_poems_count=20
novelty_del=10
df = get_dataframe('data/', min_poems_count=min_poems_count, novelty_del=novelty_del)
df.sort_values(by='time', inplace=True)

In [363]:
df.to_csv('datasets/poems_150_{}_{}.csv'.format(min_poems_count, novelty_del), index=False)

In [364]:
df.shape

(1024, 4)